In [1]:
import numpy as np

class KMeansCustom:
    def __init__(self, k=3, max_iter=300, tol=1e-4, random_state=42):
        self.k = k
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state

    def fit(self, X):
        rng = np.random.default_rng(self.random_state)
        n_samples = X.shape[0]

        # Randomly select k initial centroids
        initial_idx = rng.choice(n_samples, size=self.k, replace=False)
        self.centroids = X[initial_idx].copy()

        for _ in range(self.max_iter):
            old_centroids = self.centroids.copy()

            # Assign step
            distances = np.linalg.norm(X[:, None, :] - self.centroids[None, :, :], axis=2)
            self.labels = np.argmin(distances, axis=1)

            # Update step
            for j in range(self.k):
                cluster_points = X[self.labels == j]
                if len(cluster_points) > 0:
                    self.centroids[j] = cluster_points.mean(axis=0)

            # Check convergence
            shift = np.linalg.norm(self.centroids - old_centroids)
            if shift < self.tol:
                break

        return self.labels

    def get_centroids(self):
        return self.centroids

    def predict(self, X):
        distances = np.linalg.norm(X[:, None, :] - self.centroids[None, :, :], axis=2)
        return np.argmin(distances, axis=1)


In [2]:
import numpy as np

def ascii_scatter(X, labels, filename=None):
    x = X[:, 0]
    y = X[:, 1]

    grid_size = 40
    x_norm = (x - x.min()) / (x.max() - x.min()) * (grid_size - 1)
    y_norm = (y - y.min()) / (y.max() - y.min()) * (grid_size - 1)

    canvas = [[" " for _ in range(grid_size)] for _ in range(grid_size)]
    symbols = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    for xi, yi, lbl in zip(x_norm.astype(int), y_norm.astype(int), labels):
        sym = symbols[lbl % len(symbols)]
        canvas[grid_size - 1 - yi][xi] = sym

    ascii_result = "\n".join("".join(row) for row in canvas)

    if filename:
        with open(filename, "w") as f:
            f.write(ascii_result)

    return ascii_result


In [7]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import silhouette_score
import os

# from kmeans import KMeansCustom
# from visualize import ascii_scatter

# Load dataset
iris = load_iris()
X = iris.data

# Run K=4 (final deliverables)
model_k4 = KMeansCustom(k=4)
labels_k4 = model_k4.fit(X)
centroids_k4 = model_k4.get_centroids()

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Save centroids
with open("output/centroids_k4.txt", "w") as f:
    f.write(str(centroids_k4))

# Compute Silhouette Score
sil_score = silhouette_score(X, labels_k4)

with open("output/silhouette_k4.txt", "w") as f:
    f.write("Silhouette Score (K=4): " + str(sil_score))

# ASCII Visualization
ascii_plot = ascii_scatter(X, labels_k4, filename="output/ascii_scatter_k4.txt")

print("\nK=4 Centroids:\n", centroids_k4)
print("\nSilhouette Score (K=4):", sil_score)
print("\nASCII Scatter Plot:\n")
print(ascii_plot)


K=4 Centroids:
 [[6.2525     2.855      4.815      1.625     ]
 [5.006      3.428      1.462      0.246     ]
 [5.53214286 2.63571429 3.96071429 1.22857143]
 [6.9125     3.1        5.846875   2.13125   ]]

Silhouette Score (K=4): 0.49805050499728737

ASCII Scatter Plot:

               1                        
                                        
                                        
                                        
             1                          
         1                              
                                        
                1                       
                                        
           1                            
        1      1                    3  3
                                        
        1 11                            
   1  11                       3        
                                        
       111   1                          
                                        
   1 1 111 1      0 33        